# 2. Stereo Vision Calibration

Reads the image pairs saved by `ag-cam-tools calibration-capture` from
`stereoLeft/` and `stereoRight/`, detects checkerboard corners in each pair,
and runs OpenCV stereo calibration.

Outputs intrinsics, extrinsics, and rectification maps to `calib_result/`
for use in `3.Depthmap_with_Tuning_Bar.ipynb`.

**This notebook has no camera dependency** --- all processing is offline.

---
**Camera: Lucid PHD IMX273 --- 40 mm baseline, 3 mm FL**

Image resolution is auto-detected from the captured files (supports both
full resolution 1440x1080 and 2:1 binned 720x540).

The baseline distance is not set here; it is computed from the calibration
images and stored in the exported extrinsics (translation vector `T`).

### Setup (one-time)
```bash
cd calibration
python3 -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
python -m ipykernel install --user --name agrippa-calibration --display-name "agrippa-calibration"
```

Then select the **agrippa-calibration** kernel in Jupyter before running.

## Dependencies

In [ ]:
%pip install -r requirements.txt -q

In [1]:
import glob
import os
import cv2
import numpy as np

## Configuration

If you used a different checkerboard, update `rows`, `columns`, and
`square_size` to match.

Image size is auto-detected from the first loaded image.

In [ ]:
# Checkerboard inner-corner count and physical square size (cm).
# Inner corners = squares - 1 in each dimension.
# For an 18x25 square board, inner corners are 17x24.
rows        = 17
columns     = 24
square_size = 0.75  # cm

# 3D object points for one board view (Z=0 plane).
objp = np.zeros((rows * columns, 3), np.float32)
objp[:, :2] = np.indices((rows, columns)).T.reshape(-1, 2)
objp *= square_size

# Subpixel refinement criteria.
corner_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.01)

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# Scan for calibration session folders (contain stereoLeft/).
_sessions = sorted(
    [d for d in os.listdir(".")
     if os.path.isdir(d) and d.startswith("calibration_")
     and os.path.isdir(os.path.join(d, "stereoLeft"))],
    reverse=True,  # most recent first
)

if not _sessions:
    raise FileNotFoundError(
        "No calibration session folders found in the current directory.\n"
        "Run `ag-cam-tools calibration-capture` first."
    )

_session_dropdown = widgets.Dropdown(
    options=_sessions,
    value=_sessions[0],
    description="Session:",
    layout=widgets.Layout(width="500px"),
    style={"description_width": "80px"},
)

# Count images in each session for the info label.
def _session_info(name):
    n = len(glob.glob(os.path.join(name, "stereoLeft", "*.png")))
    has_calib = os.path.isdir(os.path.join(name, "calib_result"))
    status = " (calibrated)" if has_calib else ""
    return f"{n} image pairs{status}"

_info_label = widgets.Label(value=_session_info(_sessions[0]))

def _on_change(change):
    _info_label.value = _session_info(change["new"])

_session_dropdown.observe(_on_change, names="value")
display(widgets.HBox([_session_dropdown, _info_label]))

# Downstream cells read from this variable.
session_path = _session_dropdown.value

## Corner detection pass

Each image pair is loaded, then OpenCV searches for checkerboard corners
with subpixel refinement. Pairs where the board is not fully visible in
both frames are automatically discarded.

In [ ]:
object_points     = []
image_points_left = []
image_points_right = []
accepted = 0
discarded = 0

# Read the current dropdown selection (in case it changed since the picker cell ran).
session_path = _session_dropdown.value

# Resolve session path and create calib_result inside it.
session_abs = os.path.abspath(session_path)
calib_result_path = os.path.join(session_abs, "calib_result")
os.makedirs(calib_result_path, exist_ok=True)

left_glob  = os.path.join(session_abs, "stereoLeft",  "*.png")
right_glob = os.path.join(session_abs, "stereoRight", "*.png")

images_left  = sorted(glob.glob(left_glob))
images_right = sorted(glob.glob(right_glob))

if not images_left:
    raise FileNotFoundError(
        f"No images found in {left_glob}. "
        "Run `ag-cam-tools calibration-capture` first, then set session_path above."
    )

# Auto-detect image size from the first image.
first_img = cv2.imread(images_left[0])
image_size = (first_img.shape[1], first_img.shape[0])  # (width, height)

print(f"Session:    {session_abs}")
print(f"Found {len(images_left)} left / {len(images_right)} right images.")
print(f"Image size: {image_size[0]} x {image_size[1]}")
print("Starting corner detection...\n")

for img_left_path, img_right_path in zip(images_left, images_right):
    print(f"Pair {accepted + discarded:03d}  {os.path.basename(img_left_path)}")

    img_left  = cv2.imread(img_left_path,  cv2.IMREAD_UNCHANGED)
    img_right = cv2.imread(img_right_path, cv2.IMREAD_UNCHANGED)

    # Ensure both images are BGR.
    if img_left.ndim == 2:
        img_left  = cv2.cvtColor(img_left,  cv2.COLOR_GRAY2BGR)
        img_right = cv2.cvtColor(img_right, cv2.COLOR_GRAY2BGR)

    img_left  = cv2.resize(img_left,  image_size)
    img_right = cv2.resize(img_right, image_size)

    # Detect corners in both images.
    gray_l = cv2.cvtColor(img_left,  cv2.COLOR_BGR2GRAY)
    gray_r = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

    ret_l, corners_l = cv2.findChessboardCorners(gray_l, (rows, columns))
    ret_r, corners_r = cv2.findChessboardCorners(gray_r, (rows, columns))

    if not ret_l or not ret_r:
        reason = "left" if not ret_l else "right"
        if not ret_l and not ret_r:
            reason = "both"
        print(f"  -> discarded: chessboard not found in {reason}")
        discarded += 1
        continue

    # Subpixel refinement.
    cv2.cornerSubPix(gray_l, corners_l, (11, 11), (-1, -1), corner_criteria)
    cv2.cornerSubPix(gray_r, corners_r, (11, 11), (-1, -1), corner_criteria)

    object_points.append(objp)
    image_points_left.append(corners_l.reshape(-1, 2))
    image_points_right.append(corners_r.reshape(-1, 2))
    accepted += 1

print(f"\nDone. {accepted} pairs accepted, {discarded} discarded.")

## Stereo calibration

Runs `cv2.stereoCalibrate`, then `cv2.stereoRectify` and
`cv2.initUndistortRectifyMap` to produce the full set of rectification
parameters. Results are exported as individual `.npy` files to
`calib_result/`.

A reprojection error below **0.5 px** indicates a good calibration.

> **Distortion model:** the Edmund Optics #20-061 lens has 34.78% barrel
> distortion at full field. The rational model (`cv2.CALIB_RATIONAL_MODEL`)
> is used below, which fits an 8-coefficient distortion model instead of the
> default 5. This better captures the heavy radial distortion of these
> wide-angle lenses.

In [ ]:
print("Starting calibration — this may take a few minutes...")

calib_criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 100, 1e-5)
calib_flags = (cv2.CALIB_FIX_ASPECT_RATIO |
               cv2.CALIB_ZERO_TANGENT_DIST |
               cv2.CALIB_SAME_FOCAL_LENGTH |
               cv2.CALIB_RATIONAL_MODEL)

# Step 1: Stereo calibration.
rms, cam_mat_l, dist_l, cam_mat_r, dist_r, R, T, E, F = cv2.stereoCalibrate(
    object_points,
    image_points_left,
    image_points_right,
    cameraMatrix1=None, distCoeffs1=None,
    cameraMatrix2=None, distCoeffs2=None,
    imageSize=image_size,
    criteria=calib_criteria,
    flags=calib_flags)

print(f"RMS reprojection error: {rms:.4f} px")
print(f"Distortion coefficients: {dist_l.shape[1]} parameters (rational model)")
if rms > 0.5:
    print("  Warning: error > 0.5 px — consider re-capturing images from more angles.")
else:
    print("  Error within acceptable range.")

# Step 2: Stereo rectification.
R1, R2, P1, P2, Q, valid_roi_l, valid_roi_r = cv2.stereoRectify(
    cam_mat_l, dist_l,
    cam_mat_r, dist_r,
    image_size, R, T,
    flags=0)

# Step 3: Compute undistortion + rectification maps.
map_l1, map_l2 = cv2.initUndistortRectifyMap(
    cam_mat_l, dist_l, R1, P1, image_size, cv2.CV_32FC1)
map_r1, map_r2 = cv2.initUndistortRectifyMap(
    cam_mat_r, dist_r, R2, P2, image_size, cv2.CV_32FC1)

# Export all calibration results as .npy files (compatible with notebook 3).
np.save(os.path.join(calib_result_path, "cam_mats_left.npy"),           cam_mat_l)
np.save(os.path.join(calib_result_path, "cam_mats_right.npy"),          cam_mat_r)
np.save(os.path.join(calib_result_path, "dist_coefs_left.npy"),         dist_l)
np.save(os.path.join(calib_result_path, "dist_coefs_right.npy"),        dist_r)
np.save(os.path.join(calib_result_path, "rot_mat.npy"),                 R)
np.save(os.path.join(calib_result_path, "trans_vec.npy"),               T)
np.save(os.path.join(calib_result_path, "e_mat.npy"),                   E)
np.save(os.path.join(calib_result_path, "f_mat.npy"),                   F)
np.save(os.path.join(calib_result_path, "rect_trans_left.npy"),         R1)
np.save(os.path.join(calib_result_path, "rect_trans_right.npy"),        R2)
np.save(os.path.join(calib_result_path, "proj_mats_left.npy"),          P1)
np.save(os.path.join(calib_result_path, "proj_mats_right.npy"),         P2)
np.save(os.path.join(calib_result_path, "disp_to_depth_mat.npy"),       Q)
np.save(os.path.join(calib_result_path, "valid_boxes_left.npy"),        np.array(valid_roi_l))
np.save(os.path.join(calib_result_path, "valid_boxes_right.npy"),       np.array(valid_roi_r))
np.save(os.path.join(calib_result_path, "undistortion_map_left.npy"),   map_l1)
np.save(os.path.join(calib_result_path, "undistortion_map_right.npy"),  map_r1)
np.save(os.path.join(calib_result_path, "rectification_map_left.npy"),  map_l2)
np.save(os.path.join(calib_result_path, "rectification_map_right.npy"), map_r2)

print(f"\nCalibration exported to {calib_result_path}/ ({len(os.listdir(calib_result_path))} files)")
print(f"Baseline (||T||): {np.linalg.norm(T):.2f} {square_size and 'cm' or 'units'}")

## Inspect rectified output

Apply the rectification maps to the last image pair.
Corresponding points should fall on the same horizontal scanline in both
rectified images. If they don't, re-capture calibration images.

In [ ]:
import matplotlib.pyplot as plt

rect_left  = cv2.remap(img_left,  map_l1, map_l2, cv2.INTER_LINEAR)
rect_right = cv2.remap(img_right, map_r1, map_r2, cv2.INTER_LINEAR)

unrectified = np.hstack((
    cv2.cvtColor(img_left,  cv2.COLOR_BGR2RGB),
    cv2.cvtColor(img_right, cv2.COLOR_BGR2RGB),
))
rectified = np.hstack((
    cv2.cvtColor(rect_left,  cv2.COLOR_BGR2RGB),
    cv2.cvtColor(rect_right, cv2.COLOR_BGR2RGB),
))

fig, axes = plt.subplots(2, 1, figsize=(16, 12))

for ax, img, title in [
    (axes[0], unrectified, "Original stereo pair (left | right)"),
    (axes[1], rectified,   "Rectified stereo pair (left | right)"),
]:
    ax.imshow(img)
    h = img.shape[0]
    for y in range(0, h, h // 16):
        ax.axhline(y=y, color="lime", linewidth=0.5, alpha=0.6)
    ax.set_title(title)
    ax.axis("off")

plt.tight_layout()
plt.show()

print("Open 3.Depthmap_with_Tuning_Bar.ipynb to continue.")